In [ ]:
import csv
import chars2vec
import re
import numpy as np
import keras.backend as K
from tensorflow.keras import datasets, layers, models, losses, callbacks
from sklearn.model_selection import train_test_split
import statistics

Read the test text and labels

In [ ]:
texts = read_data_text('data/tsd_test_readable.csv')
spans = read_data_spans('data/tsd_test_readable.csv')

Inspect the text

In [ ]:
texts

Inspect the spans

In [ ]:
spans

Load the test data into a numpy array

In [ ]:
test_X = np.zeros(shape=(len(texts), 1024, 50))
c2v_model = chars2vec.load_model('eng_50')
for x, string in enumerate(texts):
    for y, char in enumerate(string):
            char_vect = c2v_model.vectorize_words([char])
            test_X[x][y] = [word_vect for word_vect in char_vect[0]]

In [ ]:
example = np.zeros(shape=(1, 1024, 50))
test_string = ["You wanker, you fucking egit I hope you die you a**hole"]

for x, string in enumerate(test_string):
    for y, char in enumerate(string):
            char_vect = c2v_model.vectorize_words([char])
            example[x][y] = [word_vect for word_vect in char_vect[0]]

In [ ]:
model = models.load_model(f"DeconvNet_model_300_epochs_sko")
y_pred = model.predict(test_X)

In [ ]:
def fix_word_boundaries(span, text):
    # "You fucking Moron you silly cunt" [6,7,8,9,10,11,12,13,14,15,16,28,29,30]
    # [4,5,6,7,8,9,10,11,12,13,14,15,16,28,29,30,31]
    seperated_text = []
    word = ''
    new_span = []
    current_word_span = []
    toxic_word = False
    for n, char in enumerate(text):
        if n in span:
            toxic_word = True
        if char == ' ':
            seperated_text.append(word)
            seperated_text.append(' ')
            word = ''
            if toxic_word:
                new_span.extend(current_word_span)
                current_word_span = []
                toxic_word = False
            else:
                current_word_span = []
                toxic_word = False
        else:
            word += char
            current_word_span.append(n)
        if n == len(text) - 1:
            seperated_text.append(word)
            if toxic_word:
                new_span.extend(current_word_span)
    return new_span

In [ ]:
scores = []
for x, pred in enumerate(y_pred):
    y_pred_f1_compatible = [j for j, i in enumerate(pred) if np.argmax(i) == 0]
    #y_pred_f1_compatible = fix_word_boundaries(y_pred_f1_compatible, texts[x])
    y_true_f1_compatible = spans[x]
    score = f1(y_pred_f1_compatible, y_true_f1_compatible)
    scores.append(score)

print('avg F1 %g' % statistics.mean(scores))


In [ ]:
test_pred = model.predict(example)

In [ ]:
for x, pred in enumerate(test_pred):
    char_arr = [j for j, i in enumerate(test_pred[0]) if np.argmax(i) == 0]
    print(f"text: {test_string[x]}")
    print(f"Predicted span: {char_arr}")
    print(f"Flagged text: {test_string[x][char_arr[0]:char_arr[-1]]}")
    